# SD212: Graph mining

## Saving a dataset

This notebook shows how to save your dataset as a ``Bunch`` object.

### Imports

In [1]:
from sknetwork.data import erdos_renyi # to generate fake data
from sknetwork.data import Bunch, save, from_edge_list
import pickle
import pandas as pd
import numpy as np

### Data cleaning

In [2]:
# your names
name = 'Vincent_Papegay-Vallet_Clement_Dussud_Youssef_Lahlou'

In [3]:
#### 
transfers = pd.read_csv("/Users/yousseflahlou/Desktop/TRAVAIL/Telecom/2A/SD/Graph mining/transfers.csv")
transfers.drop(['window', 'player_id', 'player_name','player_pos','player_nation2', 'market_val_amnt'], axis = 1 , inplace =True)
transfers = transfers[transfers['counter_team_id'] != 'Retired']
transfers['counter_team_id'] = transfers['counter_team_id'].astype('int32')
transfers = transfers[transfers['is_retired'] != True]
transfers = transfers[transfers['is_free'] != True]
transfers = transfers[transfers['is_loan'] != True]
transfers = transfers[transfers['is_loan_end'] != True]
transfers = transfers[transfers['dir'] != 'left']
transfers = transfers[transfers['transfer_fee_amnt']  < 225000000]
transfers = transfers[transfers['team_country'] != 'Career break']
transfers = transfers[transfers['team_country'] != 'Ban']
transfers = transfers[transfers['team_country'] != 'Unknown']
transfers = transfers[transfers['counter_team_country'] != 'Career break']
transfers = transfers[transfers['counter_team_country'] != 'Ban']
transfers = transfers[transfers['counter_team_country'] != 'Unknown']

transfers = transfers[transfers['counter_team_id'] != 515]
transfers = transfers[transfers['team_id'] != 515]
transfers.drop(['is_free', 'is_loan', 'is_loan_end', 'is_retired'], axis=1, inplace=True)
transfers['transfer_fee_amnt'].fillna(0, inplace=True)
transfers.dropna(axis=0)
transfers

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yousseflahlou/Desktop/TRAVAIL/Telecom/2A/SD/Graph mining/transfers.csv'

In [ ]:
transfers[transfers['counter_team_name'] == 'AS Livorno']


In [ ]:
transfers_young = transfers[transfers['player_age'] <= 23.0]
transfers_young = transfers_young[transfers_young['transfer_fee_amnt'] >= 1000000]
transfers_young

### Building Bunch object

In [ ]:
def to_team_dictionnary(nodes1, nodes2):
    dict_id = {}
    dict_country = {}
    for el in nodes1:
        dict_id[f'{el[0]}'] = el[1]
        dict_country[f'{el[0]}'] = el[2]
    for el in nodes2:
        dict_id[f'{el[0]}'] = el[1]
        dict_country[f'{el[0]}'] = el[2]
    return dict_id, dict_country

In [ ]:
def build_edges_list(nodes1, nodes2):
    edges = []
    for i in range(nodes1.shape[0]):
        edges.append((nodes1[i][0], nodes2[i][0], nodes1[i][1]))
    return edges

In [ ]:
teams_id1 = transfers[['team_id','team_name', 'team_country']].to_numpy()
teams_id2 = transfers[['counter_team_id','counter_team_name', 'counter_team_country']].to_numpy()
team_dictionnary_total, team_country_total = to_team_dictionnary(teams_id1, teams_id2)
clubs_id_total_1 = teams_id1.T[0]
clubs_id_total_2 = teams_id2.T[0]

teams_id1_young = transfers_young[['team_id','team_name', 'team_country']].to_numpy()
teams_id2_young = transfers_young[['counter_team_id','counter_team_name', 'counter_team_country']].to_numpy()
team_dictionnary_young, team_country_young = to_team_dictionnary(teams_id1_young, teams_id2_young)
clubs_id_young_1 = teams_id1_young.T[0]

In [ ]:
teams1 = transfers[['counter_team_name', 'transfer_fee_amnt']]
teams1 = teams1.to_numpy()
teams2 = transfers[['team_name']]
teams2 = teams2.to_numpy()

teams1_young = transfers_young[['counter_team_name', 'transfer_fee_amnt']]
teams1_young = teams1_young.to_numpy()
teams2_young = transfers_young[['team_name']]
teams2_young = teams2_young.to_numpy()


In [ ]:
l = build_edges_list(teams1, teams2)
l_young = build_edges_list(teams1_young, teams2_young)


In [ ]:
graph_total = from_edge_list(l,  directed=True, weighted=True, reindex = False)
adjacency_total = graph_total.adjacency
nodes_id_total = graph_total.names

graph_young = from_edge_list(l_young,  directed=True, weighted=True, reindex = False)
adjacency_young = graph_young.adjacency
nodes_id_young = graph_young.names
print(nodes_id_total[1057]) 

In [ ]:
# check
10**3 < adjacency_total.nnz < 10**6

In [ ]:
10**3 < adjacency_young.nnz < 10**6

In [ ]:
def recup_pays(array, df):
    l = []
    for el in array:
        if not (df.loc[df.team_name == f'{el}'].empty):
            l.append(df.loc[df.team_name == f'{el}'].iloc[0].team_country)
        elif not (df.loc[df.counter_team_name == f'{el}'].empty):
            l.append(df.loc[df.counter_team_name == f'{el}'].iloc[0].counter_team_country)
    return l

In [ ]:
coordonees = pd.read_csv('/Users/yousseflahlou/Desktop/TRAVAIL/Telecom/2A/SD/Graph mining/concap.csv')
coordonees.dropna(axis=1, inplace=True)
coordonees.loc[coordonees.CountryName == 'South Korea']

In [ ]:
def convert_pays_to_geo(l, df):
    n_l = []
    i=0
    for el in l[:-1]:
        if el == 'England':
            objet = df.loc[df.CountryName == 'United Kingdom']
        elif el == 'Korea, South':
            objet = df.loc[df.CountryName == 'South Korea']
        elif el == 'Scotland':
            objet = df.loc[df.CountryName == 'United Kingdom']
        elif el == 'Wales':
            objet = df.loc[df.CountryName == 'United Kingdom']
        elif el == 'Northern Ireland':
            objet = df.loc[df.CountryName == 'United Kingdom']
        elif (el == 'Bosnia-Herzegovina' or el == 'North Macedonia'):
            objet = df.loc[df.CountryName == 'Serbia']
        elif el == 'Hongkong':
            objet = df.loc[df.CountryName == 'China']
        elif (el == 'DR Congo' or el == 'Congo'):
            objet = df.loc[df.CountryName == 'Democratic Republic of the Congo']
        else:
            objet = df.loc[df.CountryName == f'{el}']
        x = objet.CapitalLatitude.to_numpy() + np.random.uniform(-5, 5)
        y = objet.CapitalLongitude.to_numpy() + np.random.uniform(-5, 5)
        if (x.size>0 and y.size>0):
            n_l.append([float(y), float(x)])
        else: 
            print(el)
            i+=1
    return n_l

In [ ]:
pays_list = recup_pays(nodes_id_total, transfers)
len(convert_pays_to_geo(pays_list, coordonees))

In [ ]:
dataset = Bunch()

In [ ]:
dataset.adjacency_total = adjacency_total
dataset.nodes_id_total = nodes_id_total
dataset.clubs_total = team_dictionnary_total 
dataset.country_total = team_country_total
dataset.clubs_id_total_1 = clubs_id_total_1

dataset.adjacency_young = adjacency_young 
dataset.nodes_id_young = nodes_id_young
dataset.clubs_young = team_dictionnary_young 
dataset.country_young = team_country_young
dataset.clubs_id_young_1 = clubs_id_young_1
print(dataset.adjacency_total.shape)

Provide details about the dataset (source, extraction, etc.)

In [ ]:
meta = Bunch()

In [ ]:
meta.name = 'Football transfers in major European leagues'
meta.description = 'Links connecting European football clubs together based on transfer history'
meta.source = 'https://github.com/d2ski/football-transfers-data/tree/main/dataset'
meta.date = 'April 2022'

In [ ]:
dataset.meta = meta

In [ ]:
pays_list = recup_pays(nodes_id_total, transfers)
converted = convert_pays_to_geo(pays_list, coordonees)
converted.append([0, 0])
dataset.geo_total = converted

pays_list = recup_pays(nodes_id_young, transfers)
converted_young = convert_pays_to_geo(pays_list, coordonees)
converted_young.append([0, 0])
dataset.geo_young = converted_young

In [ ]:
with open(name, 'bw') as f:
    pickle.dump(dataset, f)

In [ ]:
transfers['season'].values